# Process blog post:

Like so many others out there this quarantine has instilled a certain level of wanderlust in me. And obviously before I commit to any location I wanted to know that I'm not getting scalped on the airfare. To put my (and possibly your) mind at ease I've looked at historical airfare prices across X popular routes to get a view on the price fluctiations you can expect if you're travelling.

## Table of Contents:
0. Exec summary
1. Data collection
    - Schema and collection method
    - Sources
        - Airports & Routes
        - Flights & Prices
2. Data preparation and feature engineering
    - TBC
3. Model generation
4. Model validation
5. Output and visualizations
6. Extensions

## To do:
Updates for streamlining:
- collect regional mapping table of ICAO codes (e.g first letter K = USA)
- collect airline IATA codes
- possible additional data collection
    - aircraft codes
    - public aviation registers (# planes and type)

## Re-structuring airport and supplements

In [2]:
import numpy as np
import pandas as pd
import requests
import json
import sqlite3
from itertools import combinations_with_replacement
from collections import defaultdict
import time

In [3]:
# open all the csv files
airports = pd.read_csv("data/airports.csv")
destinations = pd.read_csv("data/destinations.csv")
basics = pd.read_csv("data/basics.csv")
frequency = pd.read_csv("data/frequency.csv")
runways = pd.read_csv("data/runway.csv")

In [5]:
# data integrity checks
airports["check_country_comparison"] = airports["Country"].str.replace(" ", "-").str.lower()
airports["check_url_country"] = airports["URL"].str.split("/").str[1]
airports["check"] = airports["check_country_comparison"] != airports["check_url_country"]

# discrepancies to check
airports[airports["check"]].groupby(["check_url_country", "check_country_comparison"]).count()

# duplicate URLs
print(f"There are { sum(airports.duplicated('URL')) } duplicated URLS")
airports[airports.duplicated("URL")]

There are 0 duplicated URLS


,Airport,Type,City,Country,IATA,ICAO,FAA,URL,check_country_comparison,check_url_country,check


In [6]:
# join key
join_key = ["Country", "Airport", "City"]
id_codes = ["IATA", "ICAO", "FAA"]


In [21]:
# flag duplicate by unique key
basics_dups = basics.groupby(join_key).count()[basics.groupby(join_key).count()["Metric"]>6]
airports["flag_duplicate_key"] = airports.duplicated(join_key)

# flag duplicate id codes in airports
airports["flag_duplicate_anycode"] = False
for code in id_codes:
    airports[f"flag_duplicate_{code}"] = airports.duplicated(code) & airports[code].notna()
    airports["flag_duplicate_anycode"] = airports["flag_duplicate_anycode"] | airports[f"flag_duplicate_{code}"]

In [22]:
# basics joins into main table to add long/lat/timezone
basics_t = (basics
            .drop_duplicates(subset=join_key+["Metric"], keep="first") # need to find better solution to this
            .set_index(join_key + ["Metric"])['Value']
            .unstack()
            .reset_index()
           )

airports_to_merge = airports[(airports["flag_duplicate_key"]==False) & (airports["flag_duplicate_anycode"]==False)]
master_airports = pd.merge(airports_to_merge, basics_t, how="left", on=join_key, suffixes=["", ""])

# check IATA, ICAO, FAA codes are the same
to_drop = list(master_airports.columns[master_airports.columns.str[:5] == "check"])
master_airports.loc[master_airports["ICAO Code"]=="\n", "ICAO Code"] = np.nan

for code in id_codes:
    master_airports[f"flag_{code}_discrepancy"] = master_airports[code].fillna("N/A") != master_airports[f"{code} Code"].fillna("N/A")
    master_airports[f"{code}_master"] = np.where(master_airports[code].isna(), master_airports[f"{code} Code"], master_airports[code])
    
    print(code, 
          sum(master_airports[f"flag_{code}_discrepancy"]), # number discrepancies
          sum((master_airports[f"flag_{code}_discrepancy"]) & (master_airports[f"{code} Code"].notna())), # of which are due to nan
          sum(master_airports[f"{code}_master"].isna()) - sum(master_airports[code].isna()) # gaps filled
         ) 
    
    to_drop.extend([f"flag_{code}_discrepancy", f"{code} Code", code])
    
# drop extra columns
master_airports.drop(columns = to_drop, inplace=True)

IATA 1402 0 0
ICAO 9254 0 0
FAA 10174 0 0


In [48]:
dest_counts = destinations.groupby(join_key).size().reset_index(name='nRoutes')
master_airports = pd.merge(master_airports, 
                           dest_counts, 
                           how="left", 
                           on=join_key)

# theres a fair number of airports without routes. Imagine this is due to being small so no regular commerical flights
# implication there is that there are charter flights or flights run by small operators going to these airports (OOS)

True

In [37]:
# reshaping the routes table
airports_selected = (master_airports[join_key + [code + "_master" for code in id_codes]]
                     .rename(columns={code + "_master": code + "_Source" for code in id_codes}))
routes = pd.merge(destinations, 
                  airports_selected, 
                  how="left", 
                  on=join_key, 
                  suffixes=["", "_Source"])

airports_selected = (airports_selected[airports_selected["IATA_Source"].notna()]
                   .rename(columns={code + "_Source": code for code in id_codes}))
routes = pd.merge(routes, 
                  airports_selected.rename(columns={code + "_Source": code for code in id_codes}),
                  how="left", 
                  left_on=["IATA"], # remember to change later to add city
                  right_on=["IATA"], 
                  suffixes=["", "_Dest"])

routes.rename(columns={code: code + "_Dest" for code in id_codes}, inplace=True)

In [44]:
# see how many routes can realistically look at prices for
len(routes[(routes["IATA_Source"].notna()) & (routes["IATA_Dest"].notna())].index)

32308

## Skyscanner flights

Purpose of this section is to explore the data structure of the Skyscanner API and create an ETL pipeline that will periodically (daily) add historical prices to an SQLlite DB.

Taking the airports and routes gather from the previous section, I now build up an updating view of the prices for those routes by airline. The value of the time series is to be able to look at how the time-till-flight (TTF) affects the price of the ticket.

In [46]:
# find the Skyscanner code for each airport

unique_key = ["Country"]
airport_locations = airports.drop_duplicates(subset = unique_key).loc[13889:, unique_key]

In [186]:
# load in file parameters

with open('param.json') as f:
    params = json.load(f)

headers = params["headers"]
url = params["url"]
delay = params["delay"]
long_delay = params["long_delay"]

In [47]:
# find all country codes in skyscanner

results = []
for country in airport_locations.values:
    querystring = {"query": country}
    response = requests.request("GET", url, headers=headers, params=querystring)
    time.sleep(delay)
    try:
        res_df = pd.DataFrame.from_dict(json.loads(response.text)["Places"])
        results.append(res_df)
    except KeyError:
        print(country)
        continue

In [64]:
# combine into a single df and write to disk
skyscanner_locs = pd.concat(results, axis=0, ignore_index=True)
skyscanner_locs["Flag_Airport"] = skyscanner_locs["CityId"] == "-sky"
skyscanner_locs.drop_duplicates(inplace=True)
skyscanner_locs.to_csv("skyscanner_places.csv", index=False, encoding='utf-8-sig')

### Search for quotes

In [162]:
dests = (skyscanner_locs
.loc[(skyscanner_locs["Flag_Airport"]) & (skyscanner_locs["PlaceName"]!="United Kingdom"), "PlaceId"]
.values)

In [228]:
# search parameters
# need to dynamically make this X days out from current day i.e. always be collecting data for next X days flights

uk_code = "UK-sky"
date_outbound = "2020-10-20"
date_inbound = "2020-11-20"
dates = [1]
combos = combinations_with_replacement(skyscanner_locs.loc[skyscanner_locs["Flag_Airport"], "PlaceId"].values, 1)
collected = defaultdict(list)
start_time = time.time()
counter = 0
rerun = []

# for dates in range:
for date in dates:
    for dest in combos:
        dest = dest[0]
        if dest == uk_code:
            continue
            
        # flexible delay to allow for max per min   
        elapsed_time = time.time() - start_time
        if counter >= 50 and elapsed_time <= 60:
            time.sleep(61-elapsed_time)
            start_time = time.time()
            counter = 0
        
        # make API call
        price_res = make_call(uk_code, dest, date_outbound, date_inbound, headers)
        counter = counter + 1
        
        # check result valid
        if "ValidationErrors" in price_res:
            continue
        
        if "message" in price_res:
            print("HALTED")
            time.sleep(long_delay)
            rerun.append((dest, date))
            continue
            
        for k, v in flatten_json(price_res).items():
            collected[k].append(v)
            
        time.sleep(0.25)
        

HALTED
HALTED
HALTED


In [249]:
# processing collected data
col_mapping = {
    "Routes": "QuoteIds",
    "Quotes": "CarrierIds"
}

dfs = []
for k, v in collected.items():
    
    df = pd.concat(v, axis=0, ignore_index=True)
    
    try:
        col = col_mapping[k]
        df[col] = df[col].apply(lambda x: [] if not isinstance(x, list) else x)
        df = explode(df, [col], fill_value="")
    except KeyError:
        continue
    finally:
        dfs.append(df)
    
    # add_to_db(df, k)

In [250]:
dfs[0]

,CarrierId,Name
0,50441,easyJet
1,881,British Airways
2,1878,Wizz Air
3,840,Air Algerie
4,1090,Ryanair
...,...,...
260,1090,Ryanair
261,1755,Turkish Airlines
262,1090,Ryanair
263,1324,KLM


In [ ]:
def add_to_db(df, k):
    












In [155]:
collected["Quotes"]

[   QuoteId  MinPrice  Direct        QuoteDateTime CarrierIds  OriginId  \
 0        1      67.0    True  2020-09-25T09:08:00     [1665]     63100   
 1        2      78.0    True  2020-09-23T18:21:00     [1665]     68866   
 2        3      61.0    True  2020-09-24T03:03:00     [1665]     63100   
 
    DestinationId        DepartureDate  
 0          56321  2020-10-01T00:00:00  
 1          63100  2020-10-01T00:00:00  
 2          63192  2020-10-01T00:00:00  ]

In [154]:
collected["Routes"]

[   OriginId  DestinationId QuoteIds  Price        QuoteDateTime
 0       838          43188      NaN    NaN                  NaN
 1       838          51572      NaN    NaN                  NaN
 2       838          56321      [1]   67.0  2020-09-25T09:08:00
 3       838          63100      [2]   78.0  2020-09-23T18:21:00
 4       838          63192      [3]   61.0  2020-09-24T03:03:00
 5       838          68866      NaN    NaN                  NaN
 6       838          97610      NaN    NaN                  NaN]

In [153]:
collected["Carriers"]

[   CarrierId     Name
 0       1665  Kam Air]

Now that the routes have been structured for mass API quering, I iterate through the combinations to find the day's flight prices. 

N.B. Automate this to happen every day at X time.

In [183]:
class APIManager:
    def __init__(self):
        pass
    
    def get(self):
        pass


def create_url(origin, dest, date_outbound, date_inbound, country, currency, locale):
    return f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browseroutes/v1.0/{country}/{currency}/{locale}/{origin}/{dest}/{date_outbound}"


def make_call(origin, dest, date_outbound, date_inbound, headers, country = "UK", currency = "GBP", locale = "en-UK"):
    url = create_url(origin, dest, date_outbound, date_inbound, country, currency, locale)
    querystring = {"inboundpartialdate":date_inbound}

    response = requests.request("GET", url, headers=headers, params=querystring)
    response_json = json.loads(response.text)
    # print(json.dumps(response_json, indent=2))
    return response_json


def flatten_json(quotes):
    for i, _ in enumerate(quotes['Quotes']):
        
        for key, val in quotes['Quotes'][i]['OutboundLeg'].items():
            quotes['Quotes'][i][key] = val
            
        del(quotes['Quotes'][i]['OutboundLeg'])
        
    # json to DataFrames
    # for each then need to enforce data types as well as rename any relevant columns 
    collected = {
        "Carriers" : pd.DataFrame.from_dict(quotes['Carriers']),
        "Places" : pd.DataFrame.from_dict(quotes['Places']),
        "Quotes" : pd.DataFrame.from_dict(quotes['Quotes']),
        "Routes" : pd.DataFrame.from_dict(quotes['Routes'])  
    }
    return collected

def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res